In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import scipy.stats as stats

In [2]:
sym_fea = pd.read_csv("2020_TF-IDF_dropdup.csv",index_col=0)
sym_fea[:2]

,Pediatric Obesity,Orthostatic Intolerance,Seizures,Muscle Weakness,Persistent Vegetative State,Chills,Sweating Sickness,Ataxia,Nocturia,Fetal Distress,...,Hypercalciuria,Chronic Pain,Hematemesis,Angina Pectoris,"Vision, Low",Muscle Hypertonia,"Hearing Loss, Functional",Breakthrough Pain,Mutism,Cerebrospinal Fluid Otorrhea
"Nevus, Intradermal",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Superinfection,0.0,0.0,0.0,0.635781,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.309646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
trans_df = pd.read_csv("trans_df.csv",index_col=0)
trans_df.head()

,UMLS name,MSH name
0,Acne Vulgaris,Acne Vulgaris
1,Adenitis,Lymphadenitis
2,Alzheimer's Disease,Alzheimer Disease
3,Amyloidosis,Amyloidosis
4,Fabry Disease,Fabry Disease


In [4]:
predict_score = pd.read_csv("model_predictAll_result.csv",index_col=0)

#提取5366个疾病在4268个中的索引
dis_index_inSym = []
sym_fea_index_lower = [s.lower() for s in sym_fea.index]

for dis in tqdm(trans_df["MSH name"]):
    index_inSym = sym_fea_index_lower.index(dis.lower())#从0开始
    dis_index_inSym.append(index_inSym)
    
print(len(dis_index_inSym))
dis_index_inSym[:5]

/home/yu/anaconda3/envs/jxx/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
100%|██████████| 5366/5366 [00:00<00:00, 58074.45it/s]

5366


[1279, 3131, 2883, 151, 2921]

In [25]:
DDA_mesh_df = pd.read_csv("DDA_mesh_df.csv",index_col=0)
print(DDA_mesh_df.shape)
DDA_mesh_df[:2]

/home/yu/anaconda3/envs/jxx/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(8876054, 3)


,diseaseId1_name,diseaseId2_name,jaccard
0,Alzheimer Disease,Colic,0.000294
1,Anaplasia,Colic,0.001859


In [26]:
scores = []

DDA_confs = [] #DDA，数据（0,1）

mesh_name_ls = list(trans_df["MSH name"])

for ite in tqdm(range(len(DDA_mesh_df))):
    disA_index = dis_index_inSym[mesh_name_ls.index(DDA_mesh_df.loc[ite,"diseaseId1_name"])]
    
    disB_index = dis_index_inSym[mesh_name_ls.index(DDA_mesh_df.loc[ite,"diseaseId2_name"])]
    
    scoreAB = predict_score.loc[disA_index*4268+disB_index]
    scoreBA = predict_score.loc[disB_index*4268+disA_index]
    scores.append(((scoreAB+scoreBA)/2).astype(np.float16))
    
    sim_score = DDA_mesh_df.loc[ite,"jaccard"].astype(np.float16)  #con_df 可能为多行，取均值
    DDA_confs.append(sim_score)
        
print(len(DDA_confs),len(DDA_confs)==len(scores))
DDA_confs[:2]

100%|██████████| 8876054/8876054 [1:11:09<00:00, 2079.08it/s] 

8876054 True


[0.0002944, 0.001859]

#### Calculate distribution difference

In [30]:
DDA_sim = []
DDA_unsim = []

for i in tqdm(range(len(DDA_confs))):
    score = list(scores[i])[0]
    if score>=0.5:
        DDA_sim.append(DDA_confs[i])
    else:
        DDA_unsim.append(DDA_confs[i])
        
print(len(DDA_sim),"--",len(DDA_unsim)) #

100%|██████████| 8876054/8876054 [00:23<00:00, 380629.65it/s]

2373846 -- 6502208


In [49]:
distribution_dif = stats.mannwhitneyu(DDA_sim,DDA_unsim,alternative='two-sided')
distribution_dif

MannwhitneyuResult(statistic=8830065697688.0, pvalue=0.0)

In [38]:
# pd.Series(DDA_sim).to_csv("DDA_sim.csv")
# pd.Series(DDA_unsim).to_csv("DDA_unsim.csv")

In [39]:
print(np.mean(DDA_sim))
print(np.mean(DDA_unsim))

0.02985
0.0205


'相似组的DDA的Jaccard相似性大，√'

In [61]:
distribution_dif = stats.mannwhitneyu([2]*100000000,[1]*100000000,alternative='two-sided')
distribution_dif

MannwhitneyuResult(statistic=1e+16, pvalue=0.0)